In [1]:
import re
import requests
import json
import math
import random
import execjs
from IPython.display import Video
import warnings
warnings.filterwarnings("ignore")

# 获取up主视频部分代码来自：https://github.com/Jack-Cherish/python-spider/tree/master/2020/bilibili
def get_all_videos():
    space_url = 'https://space.bilibili.com/8981370'
    search_url = 'https://api.bilibili.com/x/space/arc/search'
    mid = space_url.split('/')[-1]
    sess = requests.Session()
    search_headers = {
        'User-Agent':
        'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.167 Safari/537.36',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': 'application/json, text/plain, */*'
    }

    # 获取视频个数
    ps = 1
    pn = 1
    search_params = {'mid': mid, 'ps': ps, 'tid': 0, 'pn': pn}
    req = sess.get(url=search_url,
                   headers=search_headers,
                   params=search_params,
                   verify=False)
    info = json.loads(req.text)
    video_count = info['data']['page']['count']

    ps = 10
    page = math.ceil(video_count / ps)
    videos_list = []
    for pn in range(1, page + 1):
        search_params = {'mid': mid, 'ps': ps, 'tid': 0, 'pn': pn}
        req = sess.get(url=search_url,
                       headers=search_headers,
                       params=search_params,
                       verify=False)
        info = json.loads(req.text)
        vlist = info['data']['list']['vlist']
        for video in vlist:
            title = video['title']
            bvid = video['bvid']
            vurl = 'https://www.bilibili.com/video/' + bvid
            videos_list.append([title, vurl])
    return videos_list


def get_js_function(js_path, func_name, *args):
    with open(js_path, encoding='utf-8') as fp:
        js = fp.read()
        ctx = execjs.compile(js)
        return ctx.call(func_name, *args)


def get_random_video():
    url = random.choice(get_all_videos())[1]
    r = requests.get('http://api.bilibili.com/x/web-interface/view?bvid=' +
                     url.split('/')[-1])
    title = r.json()['data']['title']
    desc = r.json()['data']['desc']
    print('\033[1;31m即将播放：\033[0m', title, '\n\033[1;31m描述：\033[0m', desc)
    
    # 注意bilibiliurl11182这个参数名经常变化，需要重新抓包测试
    data = {'bilibiliurl11182': url, 'zengqiang': 'true'}

    r = requests.post(
        'https://xbeibeix.com/api/bilibili/',
        headers={'content-type': 'application/x-www-form-urlencoded'},
        data=data)
    hahaha = re.search("hahaha = '(.*?)';", r.text).group(1)
    download_url = get_js_function('bilibili.js', 'decrypt', hahaha)
    return download_url

In [2]:
url = get_random_video()
Video(url, width=1000, height=600)

即将播放： 【笑傲江湖】一万个舍不得 
描述： 《一万个舍不得》 原唱：庄心妍
填词/调音/剪辑：傲视在俗世上    每个人都有自己的【舍不得】，而【断舍离】却是人生的必修课。
黄绿合战Day.4 对阵作品：av60140098 投票传送门：https://www.bilibili.com/blackboard/activity-yellowVSgreen4th.html
